# Gawain sa Pag-uuri ng Teksto

Sa modyul na ito, magsisimula tayo sa isang simpleng gawain sa pag-uuri ng teksto gamit ang **[AG_NEWS](http://www.di.unipi.it/~gulli/AG_corpus_of_news_articles.html)** dataset: iuuri natin ang mga headline ng balita sa isa sa 4 na kategorya: World, Sports, Business, at Sci/Tech.

## Ang Dataset

Upang mai-load ang dataset, gagamitin natin ang **[TensorFlow Datasets](https://www.tensorflow.org/datasets)** API.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

# In this tutorial, we will be training a lot of models. In order to use GPU memory cautiously,
# we will set tensorflow option to grow GPU memory allocation when required.
physical_devices = tf.config.list_physical_devices('GPU') 
if len(physical_devices)>0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

dataset = tfds.load('ag_news_subset')

Maaari na nating ma-access ang mga bahagi ng pagsasanay at pagsusulit ng dataset gamit ang `dataset['train']` at `dataset['test']` ayon sa pagkakabanggit:


In [3]:
ds_train = dataset['train']
ds_test = dataset['test']

print(f"Length of train dataset = {len(ds_train)}")
print(f"Length of test dataset = {len(ds_test)}")

Length of train dataset = 120000
Length of test dataset = 7600


I-print natin ang unang 10 bagong headline mula sa ating dataset:


In [4]:
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

for i,x in zip(range(5),ds_train):
    print(f"{x['label']} ({classes[x['label']]}) -> {x['title']} {x['description']}")

3 (Sci/Tech) -> b'AMD Debuts Dual-Core Opteron Processor' b'AMD #39;s new dual-core Opteron chip is designed mainly for corporate computing applications, including databases, Web services, and financial transactions.'
1 (Sports) -> b"Wood's Suspension Upheld (Reuters)" b'Reuters - Major League Baseball\\Monday announced a decision on the appeal filed by Chicago Cubs\\pitcher Kerry Wood regarding a suspension stemming from an\\incident earlier this season.'
2 (Business) -> b'Bush reform may have blue states seeing red' b'President Bush #39;s  quot;revenue-neutral quot; tax reform needs losers to balance its winners, and people claiming the federal deduction for state and local taxes may be in administration planners #39; sights, news reports say.'
3 (Sci/Tech) -> b"'Halt science decline in schools'" b'Britain will run out of leading scientists unless science education is improved, says Professor Colin Pillinger.'
1 (Sports) -> b'Gerrard leaves practice' b'London, England (Sports Network

## Pagvektor ng Teksto

Ngayon, kailangan nating i-convert ang teksto sa **mga numero** na maaaring i-represent bilang tensors. Kung gusto natin ng word-level na representasyon, kailangan nating gawin ang dalawang bagay:

* Gumamit ng **tokenizer** para hatiin ang teksto sa **mga token**.
* Bumuo ng **bokabularyo** ng mga token na iyon.

### Paglilimita sa laki ng bokabularyo

Sa halimbawa ng AG News dataset, ang laki ng bokabularyo ay medyo malaki, higit sa 100k na salita. Sa pangkalahatan, hindi natin kailangan ang mga salitang bihirang lumabas sa teksto — iilan lamang na mga pangungusap ang magkakaroon nito, at hindi matututo ang modelo mula sa mga ito. Kaya't may saysay na limitahan ang laki ng bokabularyo sa mas maliit na bilang sa pamamagitan ng pagpasa ng argumento sa vectorizer constructor:

Ang parehong mga hakbang na iyon ay maaaring gawin gamit ang **TextVectorization** layer. I-instantiate natin ang vectorizer object, at pagkatapos ay tawagin ang `adapt` method upang dumaan sa lahat ng teksto at bumuo ng bokabularyo:


In [5]:
vocab_size = 50000
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size)
vectorizer.adapt(ds_train.take(500).map(lambda x: x['title']+' '+x['description']))

> **Tandaan** na gumagamit lamang tayo ng bahagi ng buong dataset upang makabuo ng bokabularyo. Ginagawa natin ito upang mapabilis ang oras ng pagproseso at hindi ka maghintay nang matagal. Gayunpaman, may panganib na ang ilang mga salita mula sa buong dataset ay hindi maisasama sa bokabularyo at hindi mapapansin sa panahon ng pagsasanay. Kaya, ang paggamit ng buong laki ng bokabularyo at pagtakbo sa buong dataset habang ginagawa ang `adapt` ay maaaring magpataas ng huling katumpakan, ngunit hindi nang malaki.

Ngayon, maaari na nating ma-access ang aktwal na bokabularyo:


In [6]:
vocab = vectorizer.get_vocabulary()
vocab_size = len(vocab)
print(vocab[:10])
print(f"Length of vocabulary: {vocab_size}")

['', '[UNK]', 'the', 'to', 'a', 'in', 'of', 'and', 'on', 'for']
Length of vocabulary: 5335


Gamit ang vectorizer, madali nating ma-encode ang anumang teksto sa isang set ng mga numero:


In [7]:
vectorizer('I love to play with my words')

<tf.Tensor: shape=(7,), dtype=int64, numpy=array([ 112, 3695,    3,  304,   11, 1041,    1], dtype=int64)>

## Representasyon ng Teksto gamit ang Bag-of-words

Dahil ang mga salita ay may kahulugan, minsan maaari nating maunawaan ang ibig sabihin ng isang piraso ng teksto sa pamamagitan lamang ng pagtingin sa mga indibidwal na salita, kahit na hindi natin isaalang-alang ang pagkakasunod-sunod nito sa pangungusap. Halimbawa, kapag nag-uuri ng balita, ang mga salitang tulad ng *weather* at *snow* ay malamang na tumutukoy sa *weather forecast*, habang ang mga salitang tulad ng *stocks* at *dollar* ay maaaring magpahiwatig ng *financial news*.

Ang **Bag-of-words** (BoW) na representasyon ng vector ay ang pinakasimple at madaling maunawaan na tradisyunal na representasyon ng vector. Ang bawat salita ay nauugnay sa isang index ng vector, at ang isang elemento ng vector ay naglalaman ng bilang ng paglitaw ng bawat salita sa isang partikular na dokumento.

![Larawan na nagpapakita kung paano kinakatawan ang bag-of-words vector sa memorya.](../../../../../translated_images/tl/bag-of-words-example.606fc1738f1d7ba9.webp) 

> **Note**: Maaari mo ring isipin ang BoW bilang kabuuan ng lahat ng one-hot-encoded vectors para sa bawat indibidwal na salita sa teksto.

Narito ang isang halimbawa kung paano bumuo ng representasyon ng bag-of-words gamit ang Scikit Learn python library:


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
sc_vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
sc_vectorizer.fit_transform(corpus)
sc_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

Maaari rin nating gamitin ang Keras vectorizer na ating tinukoy sa itaas, na nagko-convert ng bawat numero ng salita sa isang one-hot encoding at pinagsasama-sama ang lahat ng mga vector na iyon:


In [9]:
def to_bow(text):
    return tf.reduce_sum(tf.one_hot(vectorizer(text),vocab_size),axis=0)

to_bow('My dog likes hot dogs on a hot day.').numpy()

array([0., 5., 0., ..., 0., 0., 0.], dtype=float32)

> **Tandaan**: Maaaring magulat ka na ang resulta ay naiiba mula sa naunang halimbawa. Ang dahilan nito ay sa halimbawa ng Keras, ang haba ng vector ay tumutugma sa laki ng bokabularyo, na binuo mula sa buong AG News dataset, samantalang sa halimbawa ng Scikit Learn, binuo natin ang bokabularyo mula sa sample na teksto nang direkta.


## Pagsasanay sa BoW classifier

Ngayon na natutunan na natin kung paano bumuo ng bag-of-words na representasyon ng ating teksto, magtuturo tayo ng isang classifier na gagamit nito. Una, kailangan nating i-convert ang ating dataset sa isang bag-of-words na representasyon. Magagawa ito gamit ang `map` function sa ganitong paraan:


In [11]:
batch_size = 128

ds_train_bow = ds_train.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)
ds_test_bow = ds_test.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)

Ngayon, mag-define tayo ng isang simpleng classifier neural network na naglalaman ng isang linear layer. Ang input size ay `vocab_size`, at ang output size ay tumutukoy sa bilang ng mga klase (4). Dahil nagso-solve tayo ng isang classification task, ang huling activation function ay **softmax**:


In [12]:
model = keras.models.Sequential([
    keras.layers.Dense(4,activation='softmax',input_shape=(vocab_size,))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train_bow,validation_data=ds_test_bow)

938/938 [==============================] - 66s 70ms/step - loss: 0.6144 - acc: 0.8427 - val_loss: 0.4416 - val_acc: 0.8697


Dahil mayroon tayong 4 na klase, ang isang accuracy na higit sa 80% ay maituturing na magandang resulta.

## Pagsasanay ng isang classifier bilang isang network

Dahil ang vectorizer ay isa ring Keras layer, maaari tayong magtukoy ng isang network na kasama ito, at sanayin ito mula simula hanggang dulo. Sa ganitong paraan, hindi na natin kailangang i-vectorize ang dataset gamit ang `map`, maaari na lang nating ipasa ang orihinal na dataset sa input ng network.

> **Note**: Kakailanganin pa rin nating mag-apply ng mga mapa sa ating dataset upang i-convert ang mga field mula sa mga diksyunaryo (tulad ng `title`, `description`, at `label`) patungo sa mga tuple. Gayunpaman, kapag naglo-load ng data mula sa disk, maaari tayong bumuo ng isang dataset na may kinakailangang istruktura mula sa simula.


In [13]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

inp = keras.Input(shape=(1,),dtype=tf.string)
x = vectorizer(inp)
x = tf.reduce_sum(tf.one_hot(x,vocab_size),axis=1)
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
model.summary()

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 tf.one_hot (TFOpLambda)     (None, None, 5335)        0         
                                                                 
 tf.math.reduce_sum (TFOpLam  (None, 5335)             0         
 bda)                                                            
                                                                 
 dense_2 (Dense)             (None, 4)                 21344     
                                                                 
Total params: 21,344
Trainable params: 21,344
Non-trainable p

## Bigrams, trigrams at n-grams

Isang limitasyon ng bag-of-words na pamamaraan ay ang ilang mga salita ay bahagi ng mga multi-word na ekspresyon. Halimbawa, ang salitang 'hot dog' ay may ganap na ibang kahulugan kumpara sa mga salitang 'hot' at 'dog' sa ibang mga konteksto. Kung palaging ire-represent natin ang mga salitang 'hot' at 'dog' gamit ang parehong mga vector, maaaring malito ang ating modelo.

Upang tugunan ito, madalas na ginagamit ang **n-gram representations** sa mga pamamaraan ng klasipikasyon ng dokumento, kung saan ang dalas ng bawat salita, bi-word, o tri-word ay isang mahalagang tampok para sa pagsasanay ng mga classifier. Sa bigram representations, halimbawa, idinadagdag natin ang lahat ng pares ng salita sa bokabularyo, bukod pa sa mga orihinal na salita.

Narito ang isang halimbawa kung paano bumuo ng bigram bag of word representation gamit ang Scikit Learn:


In [14]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

Ang pangunahing kahinaan ng n-gram na pamamaraan ay ang mabilis na paglaki ng laki ng bokabularyo. Sa praktika, kailangan nating pagsamahin ang n-gram na representasyon sa isang teknik para sa pagbawas ng dimensyon, tulad ng *embeddings*, na tatalakayin natin sa susunod na yunit.

Upang magamit ang n-gram na representasyon sa ating **AG News** dataset, kailangan nating ipasa ang `ngrams` na parameter sa ating `TextVectorization` constructor. Ang haba ng bigram na bokabularyo ay **mas malaki nang husto**, sa ating kaso, ito ay higit sa 1.3 milyong mga token! Kaya't makatuwiran na limitahan din ang bigram na mga token sa isang makatwirang bilang.

Maaari nating gamitin ang parehong code tulad ng nasa itaas upang sanayin ang classifier, subalit, ito ay magiging napaka-inepisyente sa memorya. Sa susunod na yunit, sasanayin natin ang bigram classifier gamit ang embeddings. Sa ngayon, maaari kang mag-eksperimento sa pagsasanay ng bigram classifier sa notebook na ito at tingnan kung makakakuha ka ng mas mataas na accuracy.


## Awtomatikong Pagkalkula ng BoW Vectors

Sa halimbawa sa itaas, manu-mano nating kinalkula ang BoW vectors sa pamamagitan ng pagdaragdag ng one-hot encodings ng bawat salita. Gayunpaman, pinapayagan na tayo ng pinakabagong bersyon ng TensorFlow na awtomatikong kalkulahin ang BoW vectors sa pamamagitan ng pagpasa ng `output_mode='count` na parameter sa vectorizer constructor. Ginagawa nitong mas madali ang pagdedepina at pagsasanay ng ating modelo:


In [15]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='count'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 7s 7ms/step - loss: 0.5929 - acc: 0.8486 - val_loss: 0.4168 - val_acc: 0.8772


## Term frequency - inverse document frequency (TF-IDF)

Sa BoW na representasyon, ang pagbilang ng mga salita ay may parehong timbang anuman ang salita. Gayunpaman, malinaw na ang mga madalas na salita tulad ng *a* at *in* ay mas hindi mahalaga para sa klasipikasyon kumpara sa mga espesyalisadong termino. Sa karamihan ng mga gawain sa NLP, may mga salita na mas may kaugnayan kaysa sa iba.

Ang **TF-IDF** ay nangangahulugang **term frequency - inverse document frequency**. Isa itong bersyon ng bag-of-words, kung saan sa halip na binary na 0/1 na halaga na nagpapakita ng paglitaw ng isang salita sa isang dokumento, isang floating-point na halaga ang ginagamit, na may kaugnayan sa dalas ng paglitaw ng salita sa corpus.

Mas pormal, ang timbang $w_{ij}$ ng isang salita $i$ sa dokumento $j$ ay tinutukoy bilang:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
kung saan
* $tf_{ij}$ ay ang bilang ng paglitaw ng $i$ sa $j$, ibig sabihin, ang BoW na halaga na nakita natin dati
* $N$ ay ang bilang ng mga dokumento sa koleksyon
* $df_i$ ay ang bilang ng mga dokumento na naglalaman ng salita $i$ sa buong koleksyon

Ang halaga ng TF-IDF na $w_{ij}$ ay tumataas nang proporsyonal sa bilang ng beses na lumitaw ang isang salita sa isang dokumento at nababawasan batay sa bilang ng mga dokumento sa corpus na naglalaman ng salita, na tumutulong upang maiakma ang katotohanan na may mga salitang mas madalas lumitaw kaysa sa iba. Halimbawa, kung ang salita ay lumitaw sa *bawat* dokumento sa koleksyon, $df_i=N$, at $w_{ij}=0$, at ang mga terminong iyon ay ganap na hindi papansinin.

Madali kang makakagawa ng TF-IDF vectorization ng teksto gamit ang Scikit Learn:


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

Sa Keras, ang `TextVectorization` layer ay maaaring awtomatikong mag-compute ng TF-IDF frequencies sa pamamagitan ng pagpasa ng `output_mode='tf-idf'` na parameter. Ulitin natin ang code na ginamit natin sa itaas upang makita kung ang paggamit ng TF-IDF ay nagpapataas ng accuracy:


In [17]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='tf-idf'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 12s 12ms/step - loss: 0.4197 - acc: 0.8662 - val_loss: 0.3432 - val_acc: 0.8849


## Konklusyon

Kahit na nagbibigay ang mga representasyon ng TF-IDF ng mga timbang sa dalas ng iba't ibang salita, hindi nito kayang ipakita ang kahulugan o pagkakasunod-sunod. Tulad ng sinabi ng kilalang lingguwistang si J. R. Firth noong 1935, "Ang kumpletong kahulugan ng isang salita ay palaging nakabatay sa konteksto, at walang pag-aaral ng kahulugan na hiwalay sa konteksto ang maaaring ituring na seryoso." Matututuhan natin kung paano makuha ang impormasyong nakabatay sa konteksto mula sa teksto gamit ang language modeling sa mga susunod na bahagi ng kurso.



---

**Paunawa**:  
Ang dokumentong ito ay isinalin gamit ang AI translation service na [Co-op Translator](https://github.com/Azure/co-op-translator). Bagama't sinisikap naming maging tumpak, tandaan na ang mga awtomatikong pagsasalin ay maaaring maglaman ng mga pagkakamali o hindi pagkakatugma. Ang orihinal na dokumento sa kanyang katutubong wika ang dapat ituring na opisyal na sanggunian. Para sa mahalagang impormasyon, inirerekomenda ang propesyonal na pagsasalin ng tao. Hindi kami mananagot sa anumang hindi pagkakaunawaan o maling interpretasyon na maaaring magmula sa paggamit ng pagsasaling ito.
